In [1]:
import nltk
import glob
import os
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.wrappers import FastText
import pandas as pd
import re
import pymorphy2
import json

## Настройки

In [2]:
# Путь к файлу с текстом
corpora_path = './'

# Путь к модели эмбеддингов
# model_path = './models/news.model.bin'  # маленький корпус
# model_path = '../models/ruwikiruscorpora.model.bin'  # большой корпус
model_path = 'rutenten11_8.bin'  # большой корпус со стоп-словами

EMBEDDING_SIZE = 100  # размер эмбеддинга
MAX_SEQ_LENGTH = 30  # максимальное количество слов в предложении
N_BATCH = 128  # размер батча

# Путь к папке с предобученной моделью
# rutent2_2: Presicion: 0.9664723032069971, Recall: 0.9477445997458704, F-score: 0.9570168404170009
# rutent2_3: Presicion: 0.9608867775138559, Recall: 0.9637865311308768, F-score: 0.9623344699072238
model_folder = 'rutent2_3'

# Настройки сети
LEARNING_RATE = .0001

# Путь до словаря антонимов
ANTONYMS_PATH = 'antonyms.txt'

POSITIVE_ANSWER_THRESHOLD = 0.5

In [90]:
# Текст
SENTENCE = "Наша забастовка не от хорошей жизни: не главное, что конкурс и аукцион выиграла Лагуна; главное, что многие будут закупаться вовсе не дешевыми бумагами."

## Загрузка эмбеддингов

In [4]:
# Для первого и второго корпуса
# embeddings_vectors = KeyedVectors.load_word2vec_format(model_path, binary=True)

# Для третьего корпуса
embeddings_vectors = FastText.load_fasttext_format(model_path)

In [5]:
# # Преобразуем объект EuclidianKeyedVectors в dict (для первого и второго корпуса)

# embeddings_dict = {}
# for key in embeddings_vectors.vocab:
#     raw_word = key.split("_")[0]
#     embeddings_dict[raw_word] = embeddings_vectors[key]

In [6]:
# Для третьего корпуса
embeddings_dict = embeddings_vectors

In [7]:
# Размер эмбеддинга
EMBEDDING_SIZE = len(embeddings_dict['не'])

# Вектора, обозначающие является ли слово cue или нет
CUE_VECTOR = np.ones(EMBEDDING_SIZE)
NOT_CUE_VECTOR = np.zeros(EMBEDDING_SIZE)

# Пустой вектор
ZERO_VECTOR = np.zeros(EMBEDDING_SIZE)

## Получение эмбеддингов

In [41]:
# Слова, которых нет в словаре эмбеддингов (только для теста)
unsuccessful_embeddings = []

In [92]:
def prepare_word(word):
    
    word = word.lower()

    # Если слово содержит цифры или другие посторонние символы, выкинем его
    pattern = re.compile("[^а-яА-Яё]")
    if pattern.match(word) != None:
        return False
    
    return word

In [93]:
'''
Для предложения data создает матрицу эмбеддингов и меток, где номер строки отвечает номеру слова.
'''

def generate_batch(data):

    # Матрица эмбеддингов, номер строки -- это номер слова
    embeddings = []

    # Cue-матрица, номер строки -- это номер слова
    cues = []
    
    # Текст предложения
    text = ""
    
    # Сколько слов получилось по факту (после выкидывания неудавшихся эмбеддингов)
    n_words = 0
    
    for word_orig in nltk.word_tokenize(data):
        
        word = prepare_word(word_orig)
        if not word:
            continue
        
        try:
            # Записываем в матрицу эмбеддингов
            embeddings.append(embeddings_dict[word])

            if (word == 'не' or word == 'нет'):
                cue_vector = CUE_VECTOR
            else:
                cue_vector = NOT_CUE_VECTOR

            # Записываем в матрицу cue
            cues.append(cue_vector)

            text += word_orig + " | "
            n_words += 1

        except KeyError:
            # В случае неудачи сохраняем в список неудавшихся слов
            unsuccessful_embeddings.append(word)
                
    # Padding
    for padding_index in range(MAX_SEQ_LENGTH - n_words):
        embeddings.append(ZERO_VECTOR)
        cues.append(ZERO_VECTOR)
    
    # Ограничиваем длину предложения величиной MAX_SEQ_LENGTH
    embeddings = np.array(embeddings[:MAX_SEQ_LENGTH])
    cues = np.array(cues[:MAX_SEQ_LENGTH])
    
    return np.concatenate((embeddings, cues), axis=1), text

In [94]:
# Генерируем эмбеддинги для предложения (пока оно одно)
sent_embeddings, sent_text = generate_batch(SENTENCE)
embedding_batch = np.array([sent_embeddings])
texts = np.array([sent_text])

In [95]:
np.array(embedding_batch).shape

(1, 30, 200)

In [96]:
# Слова, для которых не удалось получить эмбеддинги
set(unsuccessful_embeddings)

set()

In [97]:
X_predict = embedding_batch

In [98]:
print("X_predict shape:", X_predict.shape)

X_predict shape: (1, 30, 200)


## Predict

In [16]:
from keras.optimizers import Adagrad, Adam
from keras.models import model_from_json
from sklearn.metrics import precision_score, recall_score, f1_score

/home/o_sopilniak/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/o_sopilniak/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [17]:
adam = Adam(lr=LEARNING_RATE)

In [18]:
# Загрузим модель

json_file = open(os.path.join(model_folder, 'model.json'), 'r')

loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

model.load_weights(os.path.join(model_folder, 'model.h5'))
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])  # 'adam', 'accuracy'

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 30, 200)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 30, 256)           336896    
_________________________________________________________________
dense_20 (Dense)             (None, 30, 2)             514       
_________________________________________________________________
activation_20 (Activation)   (None, 30, 2)             0         
Total params: 337,410
Trainable params: 337,410
Non-trainable params: 0
_________________________________________________________________


In [83]:
predicted_values_ = model.predict(X_predict)

words = texts[0].split(" | ")[:-1][:MAX_SEQ_LENGTH]

In [84]:
predicted_final = []

for value in predicted_values_[0]:
    if value[0] > POSITIVE_ANSWER_THRESHOLD:
        predicted_final.append([1., 0.])
    else:
        predicted_final.append([0., 1.])
        
predicted = np.array(predicted_final)[:, 0][:len(words)]

In [85]:
predicted

array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0.])

In [99]:
texts[0]

'Наша | забастовка | не | от | хорошей | жизни | не | главное | что | конкурс | и | аукцион | выиграла | Лагуна | главное | что | многие | будут | закупаться | вовсе | не | дешевыми | бумагами | '

## Антонимы

In [87]:
# Загрузим словарь антонимов
antonyms_dict = json.load(open(ANTONYMS_PATH))

In [88]:
morph = pymorphy2.MorphAnalyzer()

In [89]:
# Выведем пары слово - его синоним в контексте предложения

for i, word in enumerate(words):
    
    if predicted[i] == 1:

        try:
            antonyms = antonyms_dict[morph.parse(word)[0].normal_form.replace("ё", "е")]
            print("{}: {}".format(word, antonyms))

        except KeyError:
            print("Антоним не найден: ", word)

хороший: ['нехороший', 'дурной', 'плохой', 'дурной', 'нехороший', 'худой']
главное: ['второстепенный']
дешевыми: ['дорогой']
